# TorchSpatial Tutorial

- [I. Environment Settings](#i-environment-settings)
- [II. Data Download](#ii-data-download)
- [III. Example of Initializing a Location Encoder](#iii-example-of-initializing-a-location-encoder)
- [IV. Experiments on Benchmark Datasets](#iv-experiments-on-benchmark-datasets)

## I. Environment Settings

To set up the environment for the TorchSpatial project, follow these steps:

1. **Install Conda (if not already installed)**  
   Ensure you have [Anaconda](https://www.anaconda.com/products/distribution) or [Miniconda](https://docs.conda.io/en/latest/miniconda.html) installed on your system. You can download the installer from the respective links.

2. **Create the Conda Environment**  
   Open your terminal (or Anaconda Prompt on Windows) and navigate to the directory containing the `environment.yml` file. Use the following command to create the environment:
   ```bash
   conda env create -f environment.yml
   ```
   Alternatively, you can create an environment directly by following the `requirements.txt`.


## II. Data Download

The datasets can be downloaded from the following DOI link: [Download Data](https://figshare.com/articles/dataset/LocBench/26026798).

1. **Download the dataset from the specified URL**  
   For example, to download the Birdsnap dataset, use the following command in the terminal:
   ```bash
   wget https://figshare.com/ndownloader/files/47020978
2. **Extract the contents of the tar file and move the files to the desired folder if necessary**  
   ```bash
   tar -xvf 47020978
3. **Update the path for dataset**  
   Please navigate to `main/paths.py` and ensure that the dataset path matches the corresponding entry in `main/paths.py`.



## III. Example of Initializing a Location Encoder

In [ ]:
import torch
import numpy as np

from SpatialRelationEncoder import *
from module import *
from data_utils import *
from utils import *

In [2]:
# Specify the params for the location encoder, current spa_embed_dim is 128
params = {
    # The type of location encoder you will use, FIXED
    'spa_enc_type': 'Space2Vec-grid',
    'spa_embed_dim': 128,  # The dimension of the location embedding,TUNE
    'extent': (0, 200, 0, 200),  # Extent of the coords, FIXED
    'freq': 16,  # The number of scales, TUNE (See Equation(3) in (Mai, 2020))
    # Lambda_max, maximum scale, FIXED (See Equation(4) in (Mai, 2020))
    'max_radius': 1,
    # Lambda_min, minimum scale, TUNE (See Equation(4) in (Mai, 2020))
    'min_radius': 0.0001,
    'spa_f_act': "leakyrelu",  # Activation function, FIXED
    'freq_init': 'geometric',  # The method to make the Fourier frequency, FIXED
    'num_hidden_layer': 1,  # The number of hidden layer, TUNE
    'dropout': 0.5,  # Dropout rate, TUNE
    'hidden_dim': 512,  # Hidden embedding dimension, TUNE
    'use_layn': True,  # whether to you layer normalization, FIXED
    'skip_connection': True,  # Whether to use skip connection, FIXED
    'spa_enc_use_postmat': True,  # FIXED
    'device': 'cpu'  # The device, ‘cpu’ or ‘cuda:0’, etc
}

loc_enc = get_spa_encoder(
    train_locs=[],
    params=params,
    spa_enc_type=params['spa_enc_type'],
    spa_embed_dim=params['spa_embed_dim'],
    extent=params['extent'],
    coord_dim=2,
    frequency_num=params['freq'],
    max_radius=params['max_radius'],
    min_radius=params['min_radius'],
    f_act=params['spa_f_act'],
    freq_init=params['freq_init'],
    use_postmat=params['spa_enc_use_postmat'],
    device=params['device']).to(params['device'])

In [3]:
# synthetic coords data
batch_size, coord_dim = 10, 2
coords = np.random.randint(1, 201, size=(batch_size, coord_dim))

# coords: shape [batch_size, 2]
coords

array([[  8,  18],
       [187, 170],
       [ 62,  87],
       [169, 135],
       [ 97, 171],
       [145, 101],
       [152,   2],
       [ 54, 105],
       [ 27,  41],
       [ 53,  54]])

In [4]:
coords = np.array(coords)
coords = np.expand_dims(coords, axis=1)
loc_embeds = torch.squeeze(loc_enc(coords))

# loc_embed: shape [batch_size, spa_embed_dim]
loc_embeds

tensor([[ 0.0000, -0.1178,  2.8982,  ..., -0.0000, -0.8643, -0.0000],
        [-0.6335,  3.9401, -0.1604,  ...,  1.5705, -0.0699,  0.0000],
        [ 0.0000,  3.4673,  0.0000,  ...,  0.0000,  0.7862, -0.0000],
        ...,
        [ 0.0751, -0.0000, -0.0000,  ...,  0.0000, -0.2110, -0.0000],
        [ 0.0000,  3.2486, -0.3153,  ..., -0.0000,  1.8864,  0.0000],
        [ 0.0000,  1.2793, -0.6869,  ..., -0.4061, -0.7918, -0.4826]],
       grad_fn=<SqueezeBackward0>)

## IV. Experiments on Benchmark Datasets 

### 1. Model Evaluation

The **main script** for model evaluation and experiments is located at: `TorchSpatial/main/main.py`. Below are the **key command-line arguments** with their descriptions and default values.

##### 🛠️ General Options
```python
    parser.add_argument("--save_results", type=str, default="T", 
        help="Save the results (lon, lat, rr, acc1, acc3) to a CSV file for final evaluation."
    )
    parser.add_argument("--device", type=str, 
        default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
        help="Device to use: 'cuda' for GPU or 'cpu' for CPU."
    )
    parser.add_argument("--model_dir", type=str, default="../models/",
        help="Directory where models are stored."
    )
    parser.add_argument("--num_epochs", type=int, default=20,
        help="Number of training epochs."
    )
    parser.add_argument("--load_super_model", type=str, default="F", 
        help="Load a pretrained supervised model (T/F)."
    )
```
#### 📂 Dataset Options
```python
    parser.add_argument("--dataset", type=str, default="birdsnap", 
        choices=[
            "inat_2021", "inat_2018", "inat_2017", "birdsnap", "nabirds", 
            "yfcc", "fmow", "sustainbench_asset_index", 
            "sustainbench_under5_mort", "sustainbench_water_index", 
            "sustainbench_women_bmi", "sustainbench_women_edu", 
            "sustainbench_sanitation_index", "mosaiks_population", 
            "mosaiks_elevation", "mosaiks_forest_cover", 
            "mosaiks_nightlights"
        ],
        help="Dataset to use for the experiment."
    )
    parser.add_argument("--train_sample_ratio", type=float, default=0.01,
        help="Training dataset sample ratio for supervised learning."
    )
    parser.add_argument("--train_sample_method", type=str, default="random-fix",
        help="""Training dataset sampling method:
        - 'stratified-fix': Stratified sampling with fixed indices.
        - 'stratified-random': Stratified sampling with random indices.
        - 'random-fix': Random sampling with fixed indices.
        - 'random-random': Random sampling with random indices.
        - 'ssi-sample': Sample based on spatial self-information.
        """
    )
```
#### ⚙️ Training Hyperparameters
```python
    parser.add_argument("--lr", type=float, default=0.001, 
        help="Learning rate."
    )
    parser.add_argument("--lr_decay", type=float, default=0.98, 
        help="Learning rate decay factor."
    )
    parser.add_argument("--weight_decay", type=float, default=0.0, 
        help="Weight decay (L2 regularization)."
    )
    parser.add_argument("--dropout", type=float, default=0.5, 
        help="Dropout rate used in the feedforward neural network."
    )
    parser.add_argument("--batch_size", type=int, default=1024, 
        help="Batch size for training."
    )

```
#### 🔍 Logging and Evaluation
```python
    parser.add_argument("--log_frequency", type=int, default=50, 
        help="Frequency of logging (in batches)."
    )
    parser.add_argument("--max_num_exs_per_class", type=int, default=100, 
        help="Maximum number of examples per class."
    )
    parser.add_argument("--balanced_train_loader", type=str, default="T", 
        help="Use a balanced train loader (T/F)."
    )
    parser.add_argument("--eval_frequency", type=int, default=100, 
        help="Frequency of model evaluation (in batches)."
    )
    parser.add_argument("--unsuper_save_frequency", type=int, default=5, 
        help="Frequency of saving unsupervised models (in epochs)."
    )
    parser.add_argument("--do_epoch_save", type=str, default="F", 
        help="Save the model at each epoch (T/F)."
    )
```

To run the model, please navigate to the `TorchSpatial/main/` folder in the terminal and execute the command, for example:

```bash
python3 main.py \
    --save_results T \
    --load_super_model F \
    --spa_enc_type Sphere2Vec-sphereC \
    --meta_type ebird_meta \
    --dataset birdsnap \
    --eval_split test \
    --frequency_num 64 \
    --max_radius 1 \
    --min_radius 0.001 \
    --num_hidden_layer 1 \
    --hidden_dim 512 \
    --spa_f_act relu \
    --unsuper_lr 0.1 \
    --lr 0.001 \
    --model_dir ../models/sphere2vec_sphereC/ \
    --num_epochs 100 \
    --train_sample_ratio 1.0 \
    --device cpu


### 2. Model Hyperparameter Tuning

To tune the model hyperparameters, sample bash files are provided in the `TorchSpatial/main/run_sh` directory.


### 3. Model Fine Tuning

We have also provided several pre-trained models in the `TorchSpatial/pre_trained_models` directory. If you would like to fine-tune one of these models, please set the corresponding model path and load the model. Example Bash files can be found in `TorchSpatial/main/eva_sh`. For example:

```bash
python3 main.py \
    --save_results T \
    --load_super_model T \  # To load the pre-trained model
    --spa_enc_type Sphere2Vec-sphereC \
    --meta_type ebird_meta \
    --dataset birdsnap \
    --eval_split test \
    --frequency_num 64 \
    --max_radius 1 \
    --min_radius 0.001 \
    --num_hidden_layer 1 \
    --hidden_dim 512 \
    --spa_f_act relu \
    --unsuper_lr 0.1 \
    --lr 0.001 \
    --model_dir ../models/sphere2vec_sphereC/ \  # Path of pre-trained model 
    --num_epochs 30 \
    --train_sample_ratio 1.0 \
    --device cpu

### 4. Experiment Outputs
a. a log file recording the training progress and final results such as TorchSpatial/pre_trained_models/sphere2vec_sphereC/model_birdsnap_ebird_meta_Sphere2Vec-sphereC_inception_v3_0.0010_64_0.0010000_1_512.log
b. a corresponding saved model check point with the same name as log file but extentions as .pth.tar, such as TorchSpatial/pre_trained_models/sphere2vec_sphereC/model_birdsnap_ebird_meta_Sphere2Vec-sphereC_inception_v3_0.0010_64_0.0010000_1_512.pth.tar
c. If you set  --save_results to be T, a evaluated table will be generated TorchSpatial/eval_results/classification/eval_birdsnap_ebird_meta_test_Sphere2Vec-sphereC.csv